# Deep Reinforcement Learning - Computer Assignment 1

## Introduction to Deep Reinforcement Learning

Deep Reinforcement Learning (DRL) combines reinforcement learning with deep neural networks to solve complex decision-making problems. This assignment will cover the fundamental concepts, algorithms, and practical implementations of DRL.

### Learning Objectives
By the end of this assignment, you will understand:
1. **Markov Decision Processes (MDPs)** - The mathematical framework for decision making
2. **Value Functions** - State-value and action-value functions
3. **Policy Optimization** - Policy gradient methods and actor-critic algorithms
4. **Deep Q-Networks (DQN)** - Value-based deep RL methods
5. **Policy Gradient Methods** - Direct policy optimization
6. **Actor-Critic Methods** - Combining value and policy-based approaches

---

## Part 1: Theoretical Foundations

### 1.1 Markov Decision Process (MDP)

**Definition:**
An MDP is defined by the tuple $(S, A, P, R, \gamma)$ where:

-   **$S$**: Set of states - represents all possible situations the agent can encounter
-   **$A$**: Set of actions - all possible decisions the agent can make
-   **$P$**: Transition probability function $P(s'|s,a)$ - probability of moving to state $s'$ given current state $s$ and action $a$
-   **$R$**: Reward function $R(s,a,s')$ - immediate reward received for transitioning from state $s$ to $s'$ via action $a$
-   **$\gamma$**: Discount factor $[0,1]$ - determines the importance of future rewards

**Objective:**
The agent's goal is to find an optimal policy $\pi^*(a|s)$ that maximizes the expected cumulative reward:

$$G_t = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$$

**Intuition:**
Think of an MDP as a decision-making framework where:
-   You're in a specific situation (state)
-   You can take certain actions
-   Your action determines what happens next (probabilistically)
-   You get feedback (reward) for your choices
-   You want to maximize long-term success, not just immediate gain

---

### 1.2 Value Functions

**State Value Function:**
$$V^\pi(s) = \mathbb{E}_\pi[G_t | S_t = s]$$

**Interpretation:** The expected total reward when starting from state $s$ and following policy $\pi$. It answers: "How good is it to be in this state?"

**Action Value Function (Q-function):**
$$Q^\pi(s,a) = \mathbb{E}_\pi[G_t | S_t = s, A_t = a]$$

**Interpretation:** The expected total reward when taking action $a$ in state $s$ and then following policy $\pi$. It answers: "How good is it to take this specific action in this state?"

**Bellman Equations:**

For State Value Function:
$$V^\pi(s) = \sum_a \pi(a|s) \sum_{s',r} p(s',r|s,a)[r + \gamma V^\pi(s')]$$

For Action Value Function:
$$Q^\pi(s,a) = \sum_{s',r} p(s',r|s,a)[r + \gamma \sum_{a'} \pi(a'|s') Q^\pi(s',a')]$$

**Key Insight:** The Bellman equations express a recursive relationship - the value of a state depends on the immediate reward plus the discounted value of future states. This is the foundation of dynamic programming in RL.

## Part 2: Deep Q-Learning (DQN)

### 2.1 Q-Learning Algorithm

Q-Learning is a model-free, off-policy algorithm that learns the optimal action-value function:

**Q-Learning Update Rule:**
$$Q(s,a) \leftarrow Q(s,a) + \alpha [r + \gamma \max_{a'} Q(s',a') - Q(s,a)]$$

### 2.2 Deep Q-Network (DQN) Enhancements

**Key Innovations:**
1. **Experience Replay**: Store transitions $(s,a,r,s')$ in replay buffer
2. **Target Network**: Use separate network for target values to improve stability
3. **Double DQN**: Mitigate overestimation bias
4. **Dueling DQN**: Separate value and advantage streams

**DQN Loss Function:**
$$L(\theta) = \mathbb{E}_{(s,a,r,s') \sim D} \left[ \left( r + \gamma \max_{a'} Q(s',a';\theta^-) - Q(s,a;\theta) \right)^2 \right]$$

Where $\theta^-$ represents the target network parameters.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gym
import random
from collections import deque, namedtuple
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
class DQN(nn.Module):
    """
    Deep Q-Network implementation
    """
    def __init__(self, state_size, action_size, hidden_size=64):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

class DuelingDQN(nn.Module):
    """
    Dueling DQN implementation that separates value and advantage streams
    """
    def __init__(self, state_size, action_size, hidden_size=64):
        super(DuelingDQN, self).__init__()
        self.feature_layer = nn.Sequential(
            nn.Linear(state_size, hidden_size),
            nn.ReLU()
        )
        
        # Value stream
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        # Advantage stream
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, action_size)
        )
        
    def forward(self, x):
        features = self.feature_layer(x)
        
        value = self.value_stream(features)
        advantage = self.advantage_stream(features)
        
        # Combine value and advantage: Q(s,a) = V(s) + A(s,a) - mean(A(s,a))
        q_value = value + advantage - advantage.mean(dim=1, keepdim=True)
        return q_value

# Test the networks
state_size = 4  # Example: CartPole
action_size = 2  # Example: CartPole

dqn = DQN(state_size, action_size)
dueling_dqn = DuelingDQN(state_size, action_size)

print("Standard DQN architecture:")
print(dqn)
print("\nDueling DQN architecture:")
print(dueling_dqn)

In [ ]:
class ReplayBuffer:
    """
    Experience Replay Buffer for DQN
    """
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
        self.experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])
    
    def add(self, state, action, reward, next_state, done):
        """Add an experience to the buffer"""
        experience = self.experience(state, action, reward, next_state, done)
        self.buffer.append(experience)
    
    def sample(self, batch_size):
        """Sample a batch of experiences"""
        experiences = random.sample(self.buffer, k=batch_size)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        return (states, actions, rewards, next_states, dones)
    
    def __len__(self):
        return len(self.buffer)

# Test the replay buffer
buffer = ReplayBuffer(10000)
print(f"Replay buffer initialized with capacity: {10000}")
print(f"Current buffer size: {len(buffer)}")

In [ ]:
class DQNAgent:
    """
    DQN Agent with Experience Replay and Target Network
    """
    def __init__(self, state_size, action_size, lr=1e-3, gamma=0.99, 
                 epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01,
                 buffer_size=10000, batch_size=64, update_every=4, tau=1e-3,
                 use_double_dqn=False, use_dueling=False):
        
        self.state_size = state_size
        self.action_size = action_size
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size
        self.update_every = update_every
        self.tau = tau
        self.use_double_dqn = use_double_dqn
        
        # Neural networks
        if use_dueling:
            self.q_network = DuelingDQN(state_size, action_size).to(device)
            self.target_network = DuelingDQN(state_size, action_size).to(device)
        else:
            self.q_network = DQN(state_size, action_size).to(device)
            self.target_network = DQN(state_size, action_size).to(device)
            
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        
        # Replay buffer
        self.memory = ReplayBuffer(buffer_size)
        self.t_step = 0
        
        # Initialize target network
        self.hard_update(self.target_network, self.q_network)
    
    def step(self, state, action, reward, next_state, done):
        """Add experience to memory and learn if enough samples"""
        self.memory.add(state, action, reward, next_state, done)
        
        self.t_step = (self.t_step + 1) % self.update_every
        if self.t_step == 0:
            if len(self.memory) > self.batch_size:
                experiences = self.memory.sample(self.batch_size)
                self.learn(experiences)
    
    def act(self, state, eps=None):
        """Choose action using epsilon-greedy policy"""
        if eps is None:
            eps = self.epsilon
            
        if random.random() > eps:
            state = torch.from_numpy(state).float().unsqueeze(0).to(device)
            self.q_network.eval()
            with torch.no_grad():
                action_values = self.q_network(state)
            self.q_network.train()
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
    
    def learn(self, experiences):
        """Update value parameters using given batch of experience tuples"""
        states, actions, rewards, next_states, dones = experiences
        
        if self.use_double_dqn:
            # Double DQN: use main network to select actions, target network to evaluate
            next_actions = self.q_network(next_states).detach().argmax(1).unsqueeze(1)
            Q_targets_next = self.target_network(next_states).detach().gather(1, next_actions)
        else:
            # Standard DQN
            Q_targets_next = self.target_network(next_states).detach().max(1)[0].unsqueeze(1)
        
        # Compute target Q values
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))
        
        # Get expected Q values from main network
        Q_expected = self.q_network(states).gather(1, actions)
        
        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        # Update target network
        self.soft_update(self.q_network, self.target_network, self.tau)
        
        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters"""
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)
    
    def hard_update(self, target, source):
        """Hard update model parameters"""
        for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

# Initialize agent
agent = DQNAgent(state_size=4, action_size=2, use_dueling=True, use_double_dqn=True)
print("DQN Agent initialized successfully!")
print(f"Network architecture: {'Dueling' if hasattr(agent.q_network, 'value_stream') else 'Standard'} DQN")
print(f"Double DQN: {agent.use_double_dqn}")

## Part 3: Policy Gradient Methods

### 3.1 Policy Gradient Theorem

Instead of learning value functions, policy gradient methods directly optimize the policy parameters $\theta$.

**Policy Gradient Theorem:**
$$\nabla_\theta J(\theta) = \mathbb{E}_{\pi_\theta} \left[ \nabla_\theta \log \pi_\theta(a|s) \cdot Q^{\pi_\theta}(s,a) \right]$$

Where $J(\theta)$ is the expected return under policy $\pi_\theta$.

### 3.2 REINFORCE Algorithm

REINFORCE uses Monte Carlo sampling to estimate the policy gradient:

**REINFORCE Update:**
$$\theta_{t+1} = \theta_t + \alpha \nabla_\theta \log \pi_\theta(a_t|s_t) G_t$$

Where $G_t$ is the return from time step $t$.

### 3.3 Actor-Critic Methods

Actor-Critic combines policy gradient (actor) with value function approximation (critic):

- **Actor**: Updates policy parameters using policy gradient
- **Critic**: Updates value function parameters using TD learning

**Actor Update:**
$$\theta_{t+1} = \theta_t + \alpha_\theta \nabla_\theta \log \pi_\theta(a_t|s_t) \delta_t$$

**Critic Update:**
$$w_{t+1} = w_t + \alpha_w \delta_t \nabla_w V_w(s_t)$$

Where $\delta_t = r_t + \gamma V_w(s_{t+1}) - V_w(s_t)$ is the TD error.

In [ ]:
class PolicyNetwork(nn.Module):
    """
    Policy Network for REINFORCE
    """
    def __init__(self, state_size, action_size, hidden_size=64):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.softmax(self.fc3(x), dim=1)

class REINFORCEAgent:
    """
    REINFORCE Agent implementation
    """
    def __init__(self, state_size, action_size, lr=1e-3, gamma=0.99):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        
        # Policy network
        self.policy = PolicyNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        
        # Episode memory
        self.reset_episode()
        
    def reset_episode(self):
        """Reset episode memory"""
        self.states = []
        self.actions = []
        self.rewards = []
        self.log_probs = []
        
    def act(self, state):
        """Choose action based on policy"""
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.policy(state)
        
        # Sample action from probability distribution
        m = torch.distributions.Categorical(probs)
        action = m.sample()
        
        # Store log probability for later use
        self.log_probs.append(m.log_prob(action))
        
        return action.item()
    
    def step(self, state, action, reward):
        """Store step information"""
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
    
    def learn(self):
        """Update policy using REINFORCE algorithm"""
        # Calculate returns (discounted rewards)
        returns = []
        G = 0
        for r in reversed(self.rewards):
            G = r + self.gamma * G
            returns.insert(0, G)
        
        # Normalize returns (optional but often helps)
        returns = torch.tensor(returns).to(device)
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)
        
        # Calculate policy loss
        policy_loss = []
        for log_prob, G in zip(self.log_probs, returns):
            policy_loss.append(-log_prob * G)
        
        # Update policy
        self.optimizer.zero_grad()
        policy_loss = torch.cat(policy_loss).sum()
        policy_loss.backward()
        self.optimizer.step()
        
        # Reset episode memory
        self.reset_episode()
        
        return policy_loss.item()

# Initialize REINFORCE agent
reinforce_agent = REINFORCEAgent(state_size=4, action_size=2, lr=1e-3)
print("REINFORCE Agent initialized successfully!")
print("Policy network architecture:")
print(reinforce_agent.policy)

In [ ]:
class ValueNetwork(nn.Module):
    """
    Value Network for Actor-Critic
    """
    def __init__(self, state_size, hidden_size=64):
        super(ValueNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

class ActorCriticAgent:
    """
    Actor-Critic Agent implementation
    """
    def __init__(self, state_size, action_size, lr_actor=1e-3, lr_critic=1e-3, gamma=0.99):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        
        # Actor network (policy)
        self.actor = PolicyNetwork(state_size, action_size).to(device)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=lr_actor)
        
        # Critic network (value function)
        self.critic = ValueNetwork(state_size).to(device)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=lr_critic)
        
    def act(self, state):
        """Choose action based on policy"""
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.actor(state)
        
        # Sample action from probability distribution
        m = torch.distributions.Categorical(probs)
        action = m.sample()
        
        return action.item(), m.log_prob(action)
    
    def learn(self, state, action, reward, next_state, done, log_prob):
        """Update actor and critic using one-step TD error"""
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        next_state = torch.from_numpy(next_state).float().unsqueeze(0).to(device)
        reward = torch.tensor([reward]).float().to(device)
        done = torch.tensor([done]).float().to(device)
        
        # Critic update
        current_value = self.critic(state)
        next_value = self.critic(next_state) if not done else torch.zeros(1).to(device)
        
        # TD target and error
        td_target = reward + self.gamma * next_value * (1 - done)
        td_error = td_target - current_value
        
        # Critic loss (squared TD error)
        critic_loss = td_error.pow(2)
        
        # Actor loss (policy gradient with baseline)
        actor_loss = -log_prob * td_error.detach()
        
        # Update critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        
        # Update actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        return actor_loss.item(), critic_loss.item()

# Initialize Actor-Critic agent
ac_agent = ActorCriticAgent(state_size=4, action_size=2, lr_actor=1e-3, lr_critic=1e-3)
print("Actor-Critic Agent initialized successfully!")
print("Actor (Policy) network:")
print(ac_agent.actor)
print("\nCritic (Value) network:")
print(ac_agent.critic)

## Part 4: Practical Implementation and Comparison

### 4.1 Training Environment Setup

We'll use the CartPole environment from OpenAI Gym to demonstrate the algorithms:

- **State Space**: 4-dimensional continuous (position, velocity, angle, angular velocity)
- **Action Space**: 2 discrete actions (left, right)
- **Reward**: +1 for every step the pole stays upright
- **Episode Termination**: Pole angle > 15° or cart position > 2.4 units
- **Success Criteria**: Average reward > 195 over 100 consecutive episodes

In [ ]:
def train_dqn_agent(agent, env, n_episodes=1000, max_t=1000):
    """Train DQN agent"""
    scores = []
    scores_window = deque(maxlen=100)
    
    for i_episode in range(1, n_episodes + 1):
        state = env.reset()
        if isinstance(state, tuple):
            state = state[0]  # Handle new gym API
        score = 0
        
        for t in range(max_t):
            action = agent.act(state)
            result = env.step(action)
            
            if len(result) == 4:
                next_state, reward, done, _ = result
            else:
                next_state, reward, done, truncated, _ = result
                done = done or truncated
            
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            
            if done:
                break
                
        scores_window.append(score)
        scores.append(score)
        
        if i_episode % 100 == 0:
            print(f'Episode {i_episode}\tAverage Score: {np.mean(scores_window):.2f}\tEpsilon: {agent.epsilon:.3f}')
            
        if np.mean(scores_window) >= 195.0:
            print(f'Environment solved in {i_episode} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
            break
            
    return scores

def train_reinforce_agent(agent, env, n_episodes=1000, max_t=1000):
    """Train REINFORCE agent"""
    scores = []
    scores_window = deque(maxlen=100)
    
    for i_episode in range(1, n_episodes + 1):
        state = env.reset()
        if isinstance(state, tuple):
            state = state[0]  # Handle new gym API
        agent.reset_episode()
        score = 0
        
        for t in range(max_t):
            action = agent.act(state)
            result = env.step(action)
            
            if len(result) == 4:
                next_state, reward, done, _ = result
            else:
                next_state, reward, done, truncated, _ = result
                done = done or truncated
            
            agent.step(state, action, reward)
            state = next_state
            score += reward
            
            if done:
                break
        
        # Learn at the end of each episode
        loss = agent.learn()
        scores_window.append(score)
        scores.append(score)
        
        if i_episode % 100 == 0:
            print(f'Episode {i_episode}\tAverage Score: {np.mean(scores_window):.2f}')
            
        if np.mean(scores_window) >= 195.0:
            print(f'Environment solved in {i_episode} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
            break
            
    return scores

def train_actor_critic_agent(agent, env, n_episodes=1000, max_t=1000):
    """Train Actor-Critic agent"""
    scores = []
    scores_window = deque(maxlen=100)
    
    for i_episode in range(1, n_episodes + 1):
        state = env.reset()
        if isinstance(state, tuple):
            state = state[0]  # Handle new gym API
        score = 0
        
        for t in range(max_t):
            action, log_prob = agent.act(state)
            result = env.step(action)
            
            if len(result) == 4:
                next_state, reward, done, _ = result
            else:
                next_state, reward, done, truncated, _ = result
                done = done or truncated
            
            # Learn at each step
            actor_loss, critic_loss = agent.learn(state, action, reward, next_state, done, log_prob)
            
            state = next_state
            score += reward
            
            if done:
                break
        
        scores_window.append(score)
        scores.append(score)
        
        if i_episode % 100 == 0:
            print(f'Episode {i_episode}\tAverage Score: {np.mean(scores_window):.2f}')
            
        if np.mean(scores_window) >= 195.0:
            print(f'Environment solved in {i_episode} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
            break
            
    return scores

print("Training functions defined successfully!")

In [ ]:
# Create environment (using try-except for different gym versions)
try:
    import gym
    env_name = 'CartPole-v1'
    
    # Try creating environment with different gym versions
    try:
        env = gym.make(env_name)
        print(f"Environment '{env_name}' created successfully!")
        print(f"State space: {env.observation_space}")
        print(f"Action space: {env.action_space}")
        
        # Test environment
        state = env.reset()
        if isinstance(state, tuple):
            state = state[0]
        print(f"Initial state shape: {state.shape}")
        print(f"Sample state: {state}")
        
    except Exception as e:
        print(f"Error creating gym environment: {e}")
        print("Creating mock environment for demonstration...")
        
        class MockEnv:
            def __init__(self):
                self.observation_space = type('', (), {'shape': [4]})()
                self.action_space = type('', (), {'n': 2})()
                self.state = np.random.random(4)
                
            def reset(self):
                self.state = np.random.random(4)
                return self.state
                
            def step(self, action):
                self.state = np.random.random(4)
                reward = np.random.randint(0, 2)
                done = np.random.random() < 0.05
                return self.state, reward, done, {}
        
        env = MockEnv()
        print("Mock environment created for demonstration purposes.")
        
except ImportError:
    print("Gym not available, creating mock environment...")
    
    class MockEnv:
        def __init__(self):
            self.observation_space = type('', (), {'shape': [4]})()
            self.action_space = type('', (), {'n': 2})()
            self.state = np.random.random(4)
            
        def reset(self):
            self.state = np.random.random(4)
            return self.state
            
        def step(self, action):
            self.state = np.random.random(4)
            reward = np.random.randint(0, 2)
            done = np.random.random() < 0.05
            return self.state, reward, done, {}
    
    env = MockEnv()
    print("Mock environment created for demonstration purposes.")

In [ ]:
# Demonstration of training (short episodes for notebook demonstration)
n_demo_episodes = 50

print("=" * 60)
print("TRAINING DEMONSTRATION (50 episodes each)")
print("=" * 60)

# Initialize fresh agents for fair comparison
dqn_agent_demo = DQNAgent(state_size=4, action_size=2, use_dueling=True, use_double_dqn=True)
reinforce_agent_demo = REINFORCEAgent(state_size=4, action_size=2)
ac_agent_demo = ActorCriticAgent(state_size=4, action_size=2)

# Store results
results = {}

print("\n1. Training DQN Agent...")
dqn_scores = train_dqn_agent(dqn_agent_demo, env, n_episodes=n_demo_episodes, max_t=200)
results['DQN'] = dqn_scores

print("\n2. Training REINFORCE Agent...")
reinforce_scores = train_reinforce_agent(reinforce_agent_demo, env, n_episodes=n_demo_episodes, max_t=200)
results['REINFORCE'] = reinforce_scores

print("\n3. Training Actor-Critic Agent...")
ac_scores = train_actor_critic_agent(ac_agent_demo, env, n_episodes=n_demo_episodes, max_t=200)
results['Actor-Critic'] = ac_scores

print("\nTraining completed!")

In [ ]:
# Visualize training results
plt.figure(figsize=(15, 10))

# Plot 1: Learning curves
plt.subplot(2, 2, 1)
for algorithm, scores in results.items():
    plt.plot(scores, label=algorithm, linewidth=2)
    # Add moving average
    if len(scores) >= 10:
        moving_avg = []
        for i in range(len(scores)):
            if i < 9:
                moving_avg.append(np.mean(scores[:i+1]))
            else:
                moving_avg.append(np.mean(scores[i-9:i+1]))
        plt.plot(moving_avg, '--', alpha=0.7, linewidth=1)

plt.xlabel('Episode')
plt.ylabel('Score')
plt.title('Learning Curves Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Final performance comparison
plt.subplot(2, 2, 2)
final_scores = [np.mean(scores[-10:]) for scores in results.values()]
algorithms = list(results.keys())
colors = ['skyblue', 'lightcoral', 'lightgreen']
bars = plt.bar(algorithms, final_scores, color=colors, alpha=0.8)
plt.ylabel('Average Score (Last 10 Episodes)')
plt.title('Final Performance Comparison')
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, score in zip(bars, final_scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{score:.1f}', ha='center', va='bottom', fontweight='bold')

# Plot 3: Score distribution
plt.subplot(2, 2, 3)
for i, (algorithm, scores) in enumerate(results.items()):
    plt.hist(scores, bins=15, alpha=0.6, label=algorithm, 
             color=colors[i], density=True)
plt.xlabel('Score')
plt.ylabel('Density')
plt.title('Score Distribution')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: Statistical summary
plt.subplot(2, 2, 4)
stats_data = []
for algorithm, scores in results.items():
    stats_data.append({
        'Algorithm': algorithm,
        'Mean': np.mean(scores),
        'Std': np.std(scores),
        'Max': np.max(scores),
        'Min': np.min(scores)
    })

import pandas as pd
df_stats = pd.DataFrame(stats_data)
df_stats.set_index('Algorithm').plot(kind='bar', ax=plt.gca())
plt.title('Statistical Summary')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print detailed comparison
print("\n" + "="*60)
print("DETAILED PERFORMANCE ANALYSIS")
print("="*60)

for algorithm, scores in results.items():
    print(f"\n{algorithm}:")
    print(f"  Mean Score: {np.mean(scores):.2f} ± {np.std(scores):.2f}")
    print(f"  Max Score: {np.max(scores):.2f}")
    print(f"  Min Score: {np.min(scores):.2f}")
    print(f"  Final 10 Episodes: {np.mean(scores[-10:]):.2f}")
    
    # Check if solved (average > 195 for CartPole)
    best_avg = max([np.mean(scores[i:i+10]) for i in range(len(scores)-9)] if len(scores) >= 10 else [0])
    print(f"  Best 10-Episode Average: {best_avg:.2f}")
    print(f"  Environment Solved: {'Yes' if best_avg >= 195 else 'No'}")

## Part 5: Exercises and Questions

### Exercise 1: Theoretical Understanding

**Question 1.1**: Explain the difference between on-policy and off-policy learning. Which algorithms implemented in this notebook are on-policy and which are off-policy?

**Answer**: 
**On-policy vs. Off-policy Learning:**

The distinction lies in how data is used to update the policy.

-   **On-policy algorithms** update the policy based on actions taken by the *current* version of that same policy. The agent learns from the experience it generates while following its own strategy. It's like learning to cook by trying your own recipes and adjusting them based on how the food tastes. You learn from what you are currently doing.

-   **Off-policy algorithms** update the policy using data generated by a *different* policy. The agent can learn from past experiences (e.g., from a replay buffer) or from observing another agent. This separates data collection (exploration) from the learning of the optimal policy (exploitation). It's like learning to cook by watching a master chef's videos; you learn from their experience, not your own.

**Algorithms in this Notebook:**

-   **DQN (Deep Q-Network)** is **off-policy**. It uses a replay buffer to store past experiences, which may have been generated by older versions of the policy. The learning update samples from this buffer, so the data used for learning is not strictly from the current policy. This improves sample efficiency and stability.

-   **REINFORCE** is **on-policy**. It collects a full trajectory of states, actions, and rewards using its current policy. At the end of the episode, it uses this trajectory to update the policy. The data is then discarded, and a new trajectory is collected with the updated policy.

-   **Actor-Critic** (as implemented here) is **on-policy**. The actor (policy) generates an action, and the critic evaluates it. The updates are based on this immediate experience. The data is generated and used by the current policy, and then the process repeats.

---

**Question 1.2**: What is the exploration-exploitation dilemma in reinforcement learning? How do the three algorithms (DQN, REINFORCE, Actor-Critic) handle this dilemma?

**Answer**:
**The Exploration-Exploitation Dilemma:**

This is a fundamental challenge in reinforcement learning. The agent must make a trade-off between:
-   **Exploitation**: Taking the action it currently believes is the best to maximize immediate reward. This leverages known information.
-   **Exploration**: Taking a different, potentially suboptimal action to gather more information about the environment. This might lead to discovering a better long-term strategy.

The dilemma is that excessive exploration can lead to poor performance, while excessive exploitation can cause the agent to get stuck in a suboptimal strategy, never discovering better alternatives.

**How the Algorithms Handle It:**

-   **DQN**: Uses an **ε-greedy (epsilon-greedy) strategy**. With a probability `ε`, the agent takes a random action (exploration). With probability `1-ε`, it takes the action with the highest estimated Q-value (exploitation). Typically, `ε` starts high (e.g., 1.0) and is gradually decayed to a small value (e.g., 0.01), shifting the agent from exploration to exploitation as it learns more about the environment.

-   **REINFORCE**: Handles exploration through its **stochastic policy**. The policy network outputs a probability distribution over all possible actions. Actions are then sampled from this distribution. This means that even actions with lower probabilities have a non-zero chance of being selected, leading to natural exploration. As the policy improves, it will assign higher probabilities to better actions, but the inherent randomness ensures exploration continues.

-   **Actor-Critic**: Similar to REINFORCE, the **actor is a stochastic policy**. It outputs probabilities for each action, and actions are sampled accordingly. This inherent stochasticity ensures exploration. The critic's feedback helps refine these probabilities, but the agent will always have a chance to try different actions.

---

**Question 1.3**: Derive the policy gradient theorem starting from the performance measure $J(\theta) = \mathbb{E}_{s \sim \rho^\pi}[V^\pi(s)]$.

**Answer**:
The goal is to find the gradient of the performance measure $J(\theta)$ with respect to the policy parameters $\theta$. We start with the definition of the state-value function:
$V^\pi(s) = \mathbb{E}_\pi[G_t | S_t=s]$

The policy gradient theorem states:
$$\nabla_\theta J(\theta) \propto \sum_s d^\pi(s) \sum_a \nabla_\theta \pi_\theta(a|s) Q^\pi(s,a)$$

Here is a common derivation:
1.  Start with the gradient of the state-value function:
    $\nabla_\theta V^\pi(s) = \nabla_\theta \sum_a \pi_\theta(a|s) Q^\pi(s,a)$
    $= \sum_a [\nabla_\theta \pi_\theta(a|s) Q^\pi(s,a) + \pi_\theta(a|s) \nabla_\theta Q^\pi(s,a)]$ (Product Rule)

2.  Now expand the gradient of the Q-value function:
    $\nabla_\theta Q^\pi(s,a) = \nabla_\theta \sum_{s',r} p(s',r|s,a) [r + \gamma V^\pi(s')]$
    $= \gamma \sum_{s'} p(s'|s) \nabla_\theta V^\pi(s')$

3.  Substitute (2) back into (1):
    $\nabla_\theta V^\pi(s) = \sum_a [\nabla_\theta \pi_\theta(a|s) Q^\pi(s,a) + \pi_\theta(a|s) \gamma \sum_{s'} p(s'|s,a) \nabla_\theta V^\pi(s')]$

4.  This equation expresses a recursive relationship for the gradient. If we unroll it, we can see how the gradient at a state `s` depends on the gradients of future states. Let's define the discounted state distribution $d^\pi(s)$.
    The performance measure is $J(\theta) = V^\pi(s_0)$.
    $\nabla_\theta J(\theta) = \nabla_\theta V^\pi(s_0)$

5.  Unrolling the recursion from step 3 gives:
    $\nabla_\theta J(\theta) = \sum_{x \in S} d^\pi(x) \sum_a \nabla_\theta \pi_\theta(a|x) Q^\pi(x,a)$

6.  Now, use the **log-derivative trick**: $\nabla_\theta \pi_\theta(a|s) = \pi_\theta(a|s) \nabla_\theta \log \pi_\theta(a|s)$.
    Substitute this into the equation:
    $\nabla_\theta J(\theta) = \sum_{s \in S} d^\pi(s) \sum_a \pi_\theta(a|s) (\nabla_\theta \log \pi_\theta(a|s)) Q^\pi(s,a)$

7.  This can be expressed as an expectation:
    $\nabla_\theta J(\theta) = \mathbb{E}_{\pi_\theta} [\nabla_\theta \log \pi_\theta(A_t|S_t) Q^{\pi_\theta}(S_t, A_t)]$

This final form is the most common expression of the Policy Gradient Theorem. It tells us to increase the probability of actions that lead to higher-than-expected rewards.

---

### Exercise 2: Implementation Analysis

**Question 2.1**: Compare the memory requirements of DQN vs REINFORCE. Which algorithm requires more memory and why?

**Answer**:
**DQN requires significantly more memory than REINFORCE.**

The primary reason is the **Experience Replay Buffer** in DQN.
-   **DQN**: To improve stability and sample efficiency, DQN stores a large number of past transitions (`state`, `action`, `reward`, `next_state`, `done`) in a replay buffer. This buffer can be very large (e.g., holding 10,000 to 1,000,000 experiences). The agent then samples mini-batches from this buffer to perform learning updates. The memory footprint is dominated by this buffer.
-   **REINFORCE**: This algorithm is much more memory-efficient. It only needs to store the states, actions, and rewards for the *current episode*. Once the episode is finished, it uses this data to perform a single policy update, and then the data is discarded. The memory required is proportional to the length of one episode, which is typically much smaller than the capacity of a DQN replay buffer.

---

**Question 2.2**: Explain why we use a target network in DQN. What would happen if we removed it?

**Answer**:
**Why we use a target network:**

The target network is a crucial innovation for stabilizing the learning process in DQN. The Q-learning update involves calculating a target value: $y_t = r_t + \gamma \max_{a'} Q(s_{t+1}, a'; \theta)$.

If we use the *same* network for both estimating the current Q-value ($Q(s_t, a_t; \theta)$) and the target Q-value ($Q(s_{t+1}, a'; \theta)$), a problem arises. Every time we update the network weights $\theta$, the target value $y_t$ also changes. This is like trying to hit a moving target. The learning process can become unstable, leading to oscillations or divergence.

The **target network** solves this by providing a stable, fixed target for a period of time. It is a separate network whose weights ($\theta^-$) are a copy of the main Q-network's weights. These weights are held constant for several training steps and are only updated periodically (e.g., by copying the main network's weights every C steps, or through a slow "soft" update).

**What would happen if we removed it?**

Without the target network, the Q-learning target would be constantly shifting. This leads to several problems:
1.  **Instability**: The learning process is more likely to be unstable and may diverge. The loss can fluctuate wildly instead of smoothly converging.
2.  **Poor Performance**: The agent would have a much harder time learning an effective policy because it is chasing a non-stationary target.
3.  **Correlations**: The updates would be highly correlated with the current weights, which can lead to a feedback loop where incorrect Q-value estimates are reinforced.

---

**Question 2.3**: In the Dueling DQN architecture, why do we subtract the mean of the advantage values? What would happen if we didn't do this?

**Answer**:
**Why we subtract the mean of the advantage values:**

The core idea of Dueling DQN is to separately estimate the state-value function $V(s)$ and the action-advantage function $A(s,a)$. The Q-value is then reconstructed as:
$Q(s,a) = V(s) + A(s,a)$

However, this formula has an **identifiability problem**. Given a Q-value, we cannot uniquely determine the values of $V(s)$ and $A(s,a)$. For example, we could add a constant `c` to $V(s)$ and subtract it from all $A(s,a)$ values, and the resulting Q-value would be the same. This ambiguity can make training less stable.

To solve this, we enforce a constraint on the advantage function. By subtracting the mean of the advantages, we ensure that the average advantage for any state is zero:
$$Q(s,a) = V(s) + \left( A(s,a) - \frac{1}{|\mathcal{A}|} \sum_{a'} A(s,a') \right)$$
This forces $V(s)$ to be a good estimate of the state value, as it becomes the central point around which the advantages fluctuate. It stabilizes learning by ensuring that the advantage of the chosen action is a relative measure compared to the other actions.

**What would happen if we didn't do this?**

Without subtracting the mean, the network could learn to produce the same Q-values in many different ways. For example, it could:
-   Set $V(s)$ to zero and have all the Q-value information in $A(s,a)$.
-   Set all $A(s,a)$ to zero and have all the Q-value information in $V(s)$.

This ambiguity makes it difficult for the optimizer to know how to attribute the TD-error during backpropagation. The network might learn to change $V(s)$ when it should be changing $A(s,a)$, or vice-versa. This leads to **poorer performance and less stable training**. Subtracting the mean provides a clear separation of concerns, improving learning efficiency.

---

### Exercise 3: Experimental Design

Design and implement an experiment to compare the sample efficiency of the three algorithms. Consider:
- How would you measure sample efficiency?
- What metrics would you use?
- How would you ensure a fair comparison?

**Answer**:
**Experimental Design for Sample Efficiency:**

Sample efficiency refers to how much data (i.e., how many interactions with the environment) an agent needs to achieve a certain level of performance. An algorithm is more sample-efficient if it learns faster from fewer interactions.

**1. How to Measure Sample Efficiency:**

We can measure this by tracking the total number of environment steps (timesteps) taken by the agent. This is a more direct measure of experience than the number of episodes, as episodes can have variable lengths.

**2. Metrics to Use:**

-   **Timesteps to Threshold**: The primary metric would be the number of total environment interactions (timesteps) required to reach a predefined performance threshold (e.g., an average score of 195 over 100 episodes for CartPole). The algorithm that reaches this threshold in fewer timesteps is more sample-efficient.
-   **Area Under the Learning Curve (AUC)**: Plot the average score against the number of timesteps. A higher area under the curve indicates that the agent achieved higher scores earlier, signifying better sample efficiency.
-   **Performance after a Fixed Number of Steps**: Compare the average score of each algorithm after a fixed number of timesteps (e.g., after 50,000 steps). The algorithm with the higher score is more sample-efficient up to that point.

**3. How to Ensure a Fair Comparison:**

To ensure the comparison is fair, we must control for confounding variables:
-   **Identical Environments**: All agents must be trained on the exact same environment, initialized with the same random seed for the environment itself.
-   **Consistent Hyperparameters**: Use equivalent network architectures (e.g., same number of layers and hidden units) for all agents. Hyperparameters like learning rate and discount factor ($\gamma$) should be kept consistent or tuned optimally for each algorithm to ensure each is performing at its best.
-   **Averaging over Multiple Runs**: RL training can have high variance. To get reliable results, each experiment should be run multiple times (e.g., 5-10 runs) with different random seeds (for agent initialization and action selection). The results (e.g., timesteps to threshold) should then be averaged, and standard deviations should be reported to show the variance.
-   **Consistent Evaluation**: Use the same evaluation protocol for all agents, such as measuring the average score over the last 100 episodes.
-   **Total Timesteps**: The x-axis of all plots should be the total number of environment steps, not episodes, to account for varying episode lengths.

In [ ]:
# Exercise 3: Sample Efficiency Experiment
def sample_efficiency_experiment():
    """
    Implement your sample efficiency comparison here
    """
    # TODO: Implement sample efficiency metrics
    # Suggestions:
    # 1. Track number of environment interactions to reach threshold
    # 2. Compare area under learning curve
    # 3. Measure time to convergence
    
    pass

# Exercise 4: Hyperparameter Analysis
def hyperparameter_sensitivity_analysis():
    """
    Analyze how different hyperparameters affect performance
    """
    # TODO: Implement hyperparameter analysis
    # Suggestions:
    # 1. Learning rate sensitivity
    # 2. Network architecture comparison
    # 3. Exploration parameter tuning
    
    learning_rates = [1e-4, 1e-3, 1e-2]
    gamma_values = [0.9, 0.95, 0.99]
    
    results = {}
    
    # Example framework - implement your analysis here
    for lr in learning_rates:
        for gamma in gamma_values:
            # Create agent with specific hyperparameters
            # Train and record performance
            pass
    
    return results

# Exercise 5: Advanced DQN Variants
class PrioritizedReplayBuffer:
    """
    Implement Prioritized Experience Replay
    TODO: Complete this implementation
    """
    def __init__(self, capacity, alpha=0.6):
        self.capacity = capacity
        self.alpha = alpha
        # TODO: Implement prioritized replay buffer
        pass
    
    def add(self, experience, priority):
        # TODO: Add experience with priority
        pass
    
    def sample(self, batch_size, beta=0.4):
        # TODO: Sample based on priorities
        pass

class NoisyDQN(nn.Module):
    """
    Implement Noisy Networks for exploration
    TODO: Complete this implementation
    """
    def __init__(self, state_size, action_size, hidden_size=64):
        super(NoisyDQN, self).__init__()
        # TODO: Implement noisy linear layers
        pass
    
    def forward(self, x):
        # TODO: Implement forward pass with noise
        pass

print("Exercise templates created!")
print("TODO: Complete the implementations above")

## Part 6: Conclusions and Analysis

### 6.1 Algorithm Comparison Summary

| Algorithm | Type | Memory | Stability | Sample Efficiency | Exploration |
|-----------|------|--------|-----------|-------------------|-------------|
| **DQN** | Value-based | High (replay buffer) | High (target network) | High | ε-greedy |
| **REINFORCE** | Policy-based | Low | Low (high variance) | Low | Stochastic policy |
| **Actor-Critic** | Hybrid | Medium | Medium | Medium | Stochastic policy |

### 6.2 Key Insights

1. **DQN Advantages**:
   - Sample efficient due to experience replay
   - Stable learning with target networks
   - Good for discrete action spaces

2. **REINFORCE Advantages**:
   - Simple implementation
   - Works with continuous actions
   - Direct policy optimization

3. **Actor-Critic Advantages**:
   - Lower variance than REINFORCE
   - Online learning capability
   - Balances bias-variance tradeoff

### 6.3 When to Use Each Algorithm

- **Use DQN when**: Discrete actions, sample efficiency is important, you have memory constraints
- **Use REINFORCE when**: Simple problems, continuous actions, you need interpretable policies
- **Use Actor-Critic when**: You need balance between sample efficiency and stability

### 6.4 Advanced Topics for Further Study

1. **Advanced DQN Variants**:
   - Rainbow DQN (combines multiple improvements)
   - Distributional DQN
   - Quantile Regression DQN

2. **Advanced Policy Methods**:
   - Proximal Policy Optimization (PPO)
   - Trust Region Policy Optimization (TRPO)
   - Soft Actor-Critic (SAC)

3. **Model-Based RL**:
   - Model-Predictive Control
   - Dyna-Q
   - Model-based Policy Optimization

### 6.5 Further Reading

- **Books**:
  - "Reinforcement Learning: An Introduction" by Sutton & Barto
  - "Deep Reinforcement Learning Hands-On" by Maxim Lapan

- **Papers**:
  - DQN: "Human-level control through deep reinforcement learning" (Mnih et al., 2015)
  - Actor-Critic: "Actor-Critic Algorithms" (Konda & Tsitsiklis, 2000)
  - Policy Gradients: "Policy Gradient Methods" (Sutton et al., 1999)

## Assignment Submission Requirements

### What to Submit:

1. **This completed notebook** with:
   - All code cells executed
   - All theoretical questions answered
   - Experimental results and analysis

2. **Written Report** (2-3 pages) including:
   - Comparison of the three algorithms
   - Analysis of experimental results
   - Discussion of hyperparameter sensitivity
   - Recommendations for different scenarios

3. **Code Implementation** of at least one advanced feature:
   - Prioritized Experience Replay
   - Dueling DQN improvements
   - Custom environment implementation
   - Hyperparameter optimization

### Evaluation Criteria:

- **Theoretical Understanding (30%)**: Correct answers to theoretical questions
- **Implementation Quality (40%)**: Working code, proper documentation, clean structure
- **Experimental Analysis (20%)**: Thorough analysis of results, meaningful comparisons
- **Innovation/Extensions (10%)**: Creative improvements or additional implementations

### Submission Deadline: [Insert Date]

### Additional Notes:

- Ensure all code runs without errors
- Include clear comments and documentation
- Use proper citation for any external sources
- Submit both .ipynb and .pdf versions of the notebook

---

**Good luck with your Deep Reinforcement Learning journey!** 🚀

Remember: The key to mastering DRL is understanding the trade-offs between different algorithms and knowing when to apply each one. Practice implementing these algorithms on different environments to build intuition.